# Improvements:
- Split casual and registered - completed
- create two separate models + test combined model - completed
- undersample summer/spring - oversample fall/winter - unfinished (unsure if this is necessary)
- reevaluate minmax (fix data leak) - completed
- add tuner - completed
- work on scheduler - integrate into tuner - completed

# Pre-Processing

## Imports

In [1]:
!pip install tensorflow keras-tuner

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


In [2]:
import pandas as pd
import numpy as np
import tensorflow as tf
from sklearn.preprocessing import MinMaxScaler
from sklearn.model_selection import train_test_split
from keras.models import Sequential
from keras.layers import Dense
from keras.callbacks import EarlyStopping, LearningRateScheduler, Callback
import math
from tensorflow.keras import layers
import keras_tuner as kt
from kerastuner.tuners import Hyperband
from google.colab import drive
drive.mount('/content/drive')
my_drive_dir = '/content/drive/My Drive/Project_3/Jared-Work'

# loading in the bikes csv
bikes_df = pd.read_csv('https://raw.githubusercontent.com/byui-cse/cse450-course/master/data/bikes.csv')
bikes_mini = pd.read_csv('https://raw.githubusercontent.com/byui-cse/cse450-course/master/data/biking_holdout_test_mini.csv')
bikes_holdout = pd.read_csv('https://raw.githubusercontent.com/byui-cse/cse450-course/master/data/bikes_december.csv')

# check out the info
bikes_df.info()
# holdout_X = holdout_X.reindex(columns=X.columns)
# holdout_X = fill.na(0)

<ipython-input-2-87c61120bbd7>:11: DeprecationWarning: `import kerastuner` is deprecated, please use `import keras_tuner`.
  from kerastuner.tuners import Hyperband


Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 16637 entries, 0 to 16636
Data columns (total 12 columns):
 #   Column        Non-Null Count  Dtype  
---  ------        --------------  -----  
 0   dteday        16637 non-null  object 
 1   season        16637 non-null  int64  
 2   hr            16637 non-null  int64  
 3   holiday       16637 non-null  int64  
 4   workingday    16637 non-null  int64  
 5   weathersit    16637 non-null  int64  
 6   hum           16637 non-null  float64
 7   windspeed     16637 non-null  int64  
 8   temp_c        16637 non-null  float64
 9   feels_like_c  16637 non-null  float64
 10  casual        16637 non-null  int64  
 11  registered    16637 non-null  int64  
dtypes: float64(3), int64(8), object(1)
memory usage: 1.5+ MB


In [3]:
bikes_df.head()

,dteday,season,hr,holiday,workingday,weathersit,hum,windspeed,temp_c,feels_like_c,casual,registered
0,1/1/11,1,0,0,0,1,0.81,0,3.28,3.0014,3,13
1,1/1/11,1,1,0,0,1,0.80,0,2.34,1.9982,8,32
2,1/1/11,1,2,0,0,1,0.80,0,2.34,1.9982,5,27
3,1/1/11,1,3,0,0,1,0.75,0,3.28,3.0014,3,10
4,1/1/11,1,4,0,0,1,0.75,0,3.28,3.0014,0,1


In [4]:
bikes_mini.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 35 entries, 0 to 34
Data columns (total 10 columns):
 #   Column        Non-Null Count  Dtype  
---  ------        --------------  -----  
 0   dteday        35 non-null     object 
 1   season        35 non-null     int64  
 2   hr            35 non-null     int64  
 3   holiday       35 non-null     int64  
 4   workingday    35 non-null     int64  
 5   weathersit    35 non-null     int64  
 6   hum           35 non-null     float64
 7   windspeed     35 non-null     int64  
 8   temp_c        35 non-null     float64
 9   feels_like_c  35 non-null     float64
dtypes: float64(3), int64(6), object(1)
memory usage: 2.9+ KB


In [5]:
bikes_mini.head()

,dteday,season,hr,holiday,workingday,weathersit,hum,windspeed,temp_c,feels_like_c
0,12/1/12,4,0,0,0,1,0.81,0,4.22,3.9980
1,12/1/12,4,1,0,0,1,0.81,0,4.22,3.9980
2,12/1/12,4,2,0,0,2,0.81,0,4.22,3.9980
3,12/1/12,4,3,0,0,2,0.81,9,4.22,1.9982
4,12/1/12,4,4,0,0,1,0.81,6,4.22,3.0014


In [6]:
bikes_holdout.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 742 entries, 0 to 741
Data columns (total 10 columns):
 #   Column        Non-Null Count  Dtype  
---  ------        --------------  -----  
 0   dteday        742 non-null    object 
 1   season        742 non-null    int64  
 2   hr            742 non-null    int64  
 3   holiday       742 non-null    int64  
 4   workingday    742 non-null    int64  
 5   weathersit    742 non-null    int64  
 6   hum           742 non-null    float64
 7   windspeed     742 non-null    int64  
 8   temp_c        742 non-null    float64
 9   feels_like_c  742 non-null    float64
dtypes: float64(3), int64(6), object(1)
memory usage: 58.1+ KB


In [7]:
bikes_holdout.head()

,dteday,season,hr,holiday,workingday,weathersit,hum,windspeed,temp_c,feels_like_c
0,12/1/12,4,0,0,0,1,0.81,0,4.22,3.9980
1,12/1/12,4,1,0,0,1,0.81,0,4.22,3.9980
2,12/1/12,4,2,0,0,2,0.81,0,4.22,3.9980
3,12/1/12,4,3,0,0,2,0.81,9,4.22,1.9982
4,12/1/12,4,4,0,0,1,0.81,6,4.22,3.0014


In [8]:
# Preprocessing 
bikes_df.head()
bikes_df['dteday'] = pd.to_datetime(bikes_df['dteday'], format='%m/%d/%y')
bikes_mini['dteday'] = pd.to_datetime(bikes_mini['dteday'], format='%m/%d/%y')
bikes_holdout['dteday'] = pd.to_datetime(bikes_holdout['dteday'], format='%m/%d/%y')

# create new features - year, month, day, dayofweek
bikes_df['year'] = pd.to_datetime(bikes_df['dteday']).dt.year
bikes_df['month'] = pd.to_datetime(bikes_df['dteday']).dt.month
bikes_df['day'] = pd.to_datetime(bikes_df['dteday']).dt.day
bikes_df['dayofweek'] = pd.to_datetime(bikes_df['dteday']).dt.dayofweek

bikes_mini['year'] = pd.to_datetime(bikes_mini['dteday']).dt.year
bikes_mini['month'] = pd.to_datetime(bikes_mini['dteday']).dt.month
bikes_mini['day'] = pd.to_datetime(bikes_mini['dteday']).dt.day
bikes_mini['dayofweek'] = pd.to_datetime(bikes_mini['dteday']).dt.dayofweek

bikes_holdout['year'] = pd.to_datetime(bikes_holdout['dteday']).dt.year
bikes_holdout['month'] = pd.to_datetime(bikes_holdout['dteday']).dt.month
bikes_holdout['day'] = pd.to_datetime(bikes_holdout['dteday']).dt.day
bikes_holdout['dayofweek'] = pd.to_datetime(bikes_holdout['dteday']).dt.dayofweek

# drop dteday column
bikes_df = bikes_df.drop('dteday', axis=1)
bikes_mini = bikes_mini.drop('dteday', axis=1)
bikes_holdout = bikes_holdout.drop('dteday', axis=1)

# one hot encoding
categorical_features = ['season', 'hr', 'holiday', 'workingday', 'weathersit', 'year', 'month', 'day','dayofweek']
bikes_df = pd.get_dummies(bikes_df, columns=categorical_features, dtype=int)
bikes_mini = pd.get_dummies(bikes_mini, columns=categorical_features, dtype=int)
bikes_holdout = pd.get_dummies(bikes_holdout, columns=categorical_features, dtype=int)

# features and the target
y_casual = bikes_df['casual']
y_registered = bikes_df['registered']

# X 
X = bikes_df.drop(columns=['casual', 'registered'])

# First split to separate out the training set
X_train, X_other, y_train, y_other = train_test_split(X, bikes_df[['casual', 'registered']], test_size = 0.2, random_state = 42)

# Second split to separate out the test and validation sets
X_val, X_test, y_val, y_test = train_test_split(X_other, y_other, test_size = 0.5, random_state = 0)


# Scaling
scaler = MinMaxScaler()
X_train_scaled = scaler.fit_transform(X_train)
X_test_scaled = scaler.transform(X_test)
X_val_scaled = scaler.transform(X_val)

# Ensure bikes_mini and bikes_holdout have the same columns as X_train
bikes_mini = bikes_mini.reindex(columns = X_train.columns).fillna(0)
bikes_holdout = bikes_holdout.reindex(columns = X_train.columns).fillna(0)

# Scale bikes_mini and bikes_holdout using the scaler fitted on X_train
X_mini_scaled = scaler.transform(bikes_mini)
X_holdout_scaled = scaler.transform(bikes_holdout)

In [9]:
X_train_scaled.shape

(13309, 92)

In [10]:
y_train.head()

,casual,registered
13449,46,151
1085,4,40
14240,76,255
8370,20,109
1924,8,31


In [11]:
# bikes_df.head()
# bikes_df.to_csv('bikes_output1.csv', index=False)
#bikes_mini.to_csv('bikes_mini1.csv', index=False)

In [12]:
bikes_mini.head()

,hum,windspeed,temp_c,feels_like_c,season_1,season_2,season_3,season_4,hr_0,hr_1,...,day_29,day_30,day_31,dayofweek_0,dayofweek_1,dayofweek_2,dayofweek_3,dayofweek_4,dayofweek_5,dayofweek_6
0,0.81,0,4.22,3.9980,0.0,0.0,0.0,1,1,0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1,0
1,0.81,0,4.22,3.9980,0.0,0.0,0.0,1,0,1,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1,0
2,0.81,0,4.22,3.9980,0.0,0.0,0.0,1,0,0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1,0
3,0.81,9,4.22,1.9982,0.0,0.0,0.0,1,0,0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1,0
4,0.81,6,4.22,3.0014,0.0,0.0,0.0,1,0,0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1,0


In [13]:
bikes_holdout.head()

,hum,windspeed,temp_c,feels_like_c,season_1,season_2,season_3,season_4,hr_0,hr_1,...,day_29,day_30,day_31,dayofweek_0,dayofweek_1,dayofweek_2,dayofweek_3,dayofweek_4,dayofweek_5,dayofweek_6
0,0.81,0,4.22,3.9980,0,0.0,0.0,1,1,0,...,0,0,0,0,0,0,0,0,1,0
1,0.81,0,4.22,3.9980,0,0.0,0.0,1,0,1,...,0,0,0,0,0,0,0,0,1,0
2,0.81,0,4.22,3.9980,0,0.0,0.0,1,0,0,...,0,0,0,0,0,0,0,0,1,0
3,0.81,9,4.22,1.9982,0,0.0,0.0,1,0,0,...,0,0,0,0,0,0,0,0,1,0
4,0.81,6,4.22,3.0014,0,0.0,0.0,1,0,0,...,0,0,0,0,0,0,0,0,1,0


In [ ]:
import keras_tuner as kt
from tensorflow.keras import layers
from tensorflow.keras.callbacks import EarlyStopping, LearningRateScheduler, ModelCheckpoint
from tensorflow.keras import regularizers
import tensorflow as tf
import math

# Define the model-building function for Keras Tuner
def model_builder(hp):
    model = tf.keras.Sequential()
    n_inputs = bikes_df.shape[1] - 2  # -2 because we removed 'casual' and 'registered' columns

    # Input layer
    model.add(layers.Dense(n_inputs, input_dim=n_inputs, activation='relu'))

    # Tune the number of hidden layers
    for i in range(hp.Int('num_layers', 1, 3)):
        # Tune the number of units in the i-th hidden layer
        hp_units = hp.Int('units_' + str(i), min_value=32, max_value=256, step=32)
        model.add(layers.Dense(units=hp_units, activation='relu', kernel_regularizer=regularizers.l2(0.001)))
        model.add(layers.BatchNormalization())
        model.add(layers.Dropout(0.2))

    # Output layer
    model.add(layers.Dense(2, activation='linear'))  # output layer has 2 neurons

    # Tune the learning rate for the optimizer
    hp_learning_rate = hp.Choice('learning_rate', values=[1e-2, 1e-3, 1e-4])

    # Compile the model
    model.compile(optimizer=tf.keras.optimizers.Adam(learning_rate=hp_learning_rate),
                  loss='mean_squared_error',  # 'mean_squared_error' works with multi-output regression
                  metrics=[tf.keras.metrics.RootMeanSquaredError()])  # update metrics to handle multi-output

    return model

# scheduler
def scheduler(epoch, lr):
    if epoch < 10:
        return lr
    else:
        return lr * math.exp(-0.1)

# Define callbacks
early_stopping = EarlyStopping(monitor='val_loss', patience=10)
lr_scheduler = LearningRateScheduler(scheduler)
model_checkpoint = ModelCheckpoint('best_model.h5', monitor='val_loss', save_best_only=True)
callbacks = [early_stopping, lr_scheduler, model_checkpoint]

# Merge y_train['casual'] and y_train['registered'] into a single DataFrame
y_train_combined = pd.concat([y_train['casual'], y_train['registered']], axis=1)

# Adjust the tuner to work with the combined y_train
tuner_combined = kt.Hyperband(model_builder,
                     objective=kt.Objective('val_root_mean_squared_error', direction='min'),
                     max_epochs=300,
                     factor=3,
                     directory=my_drive_dir,
                     project_name='Model_combined')

# Perform hyperparameter tuning with the combined y_train
tuner_combined.search(X_train_scaled, y_train_combined, epochs=30, validation_split=0.2, callbacks=callbacks, batch_size=64)

Trial 56 Complete [00h 00m 13s]
val_root_mean_squared_error: 39.824100494384766

Best val_root_mean_squared_error So Far: 24.87245750427246
Total elapsed time: 00h 06m 56s

Search: Running Trial #57

Value             |Best Value So Far |Hyperparameter
2                 |2                 |num_layers
192               |256               |units_0
0.01              |0.01              |learning_rate
160               |192               |units_1
96                |96                |units_2
4                 |30                |tuner/epochs
0                 |10                |tuner/initial_epoch
2                 |3                 |tuner/bracket
0                 |3                 |tuner/round

Epoch 1/4
167/167 [==============================] - 5s 14ms/step - loss: 7767.2021 - root_mean_squared_error: 88.1272 - val_loss: 4035.4297 - val_root_mean_squared_error: 63.5162 - lr: 0.0100
Epoch 2/4
165/167 [============================>.] - ETA: 0s - loss: 1593.5327 - root_mean_squared_erro

Trial 90 Complete [00h 00m 22s]
val_root_mean_squared_error: 25.46880340576172

Best val_root_mean_squared_error So Far: 24.5953426361084
Total elapsed time: 00h 13m 24s

In [15]:
# Get the best model
best_model_combined = tuner_combined.get_best_models(num_models=1)[0]

# Merge y_train['casual'] and y_train['registered'] into a single DataFrame
y_train_combined = pd.concat([y_train['casual'], y_train['registered']], axis=1)
y_test_combined = pd.concat([y_test['casual'], y_test['registered']], axis=1)
y_val_combined = pd.concat([y_val['casual'], y_val['registered']], axis=1)

# Evaluate the model on the training data
train_mse, train_rmse = best_model_combined.evaluate(X_train_scaled, y_train_combined, verbose = 1)

# Evaluate the model on the testing data
test_mse, test_rmse = best_model_combined.evaluate(X_test_scaled, y_test_combined, verbose = 1)

# Evaluate the model on the validation data
val_mse, val_rmse = best_model_combined.evaluate(X_val_scaled, y_val_combined, verbose = 1)

52/52 [==============================] - 1s 12ms/step - loss: 931.0995 - root_mean_squared_error: 30.4829


In [ ]:
# Get predictions for training, testing and validation datasets
train_predictions = best_model_combined.predict(X_train_scaled)
test_predictions = best_model_combined.predict(X_test_scaled)
val_predictions = best_model_combined.predict(X_val_scaled)

# Extract 'casual' and 'registered' predictions
train_predictions_casual = train_predictions[:, 0]
train_predictions_registered = train_predictions[:, 1]

test_predictions_casual = test_predictions[:, 0]
test_predictions_registered = test_predictions[:, 1]

val_predictions_casual = val_predictions[:, 0]
val_predictions_registered = val_predictions[:, 1]

# Compute RMSE for 'casual' and 'registered'
train_rmse_casual = np.sqrt(mean_squared_error(y_train['casual'], train_predictions_casual))
train_rmse_registered = np.sqrt(mean_squared_error(y_train['registered'], train_predictions_registered))

test_rmse_casual = np.sqrt(mean_squared_error(y_test['casual'], test_predictions_casual))
test_rmse_registered = np.sqrt(mean_squared_error(y_test['registered'], test_predictions_registered))

val_rmse_casual = np.sqrt(mean_squared_error(y_val['casual'], val_predictions_casual))
val_rmse_registered = np.sqrt(mean_squared_error(y_val['registered'], val_predictions_registered))

# Print the RMSE scores
print(f'Train RMSE for casual: {train_rmse_casual}')
print(f'Train RMSE for registered: {train_rmse_registered}')
print(f'Test RMSE for casual: {test_rmse_casual}')
print(f'Test RMSE for registered: {test_rmse_registered}')
print(f'Validation RMSE for casual: {val_rmse_casual}')
print(f'Validation RMSE for registered: {val_rmse_registered}')

416/416 [==============================] - 1s 2ms/step - loss: 336.4983 - root_mean_squared_error: 18.2342
52/52 [==============================] - 0s 5ms/step - loss: 512.3221 - root_mean_squared_error: 22.5457
52/52 [==============================] - 0s 2ms/step - loss: 577.9160 - root_mean_squared_error: 23.9563

416/416 [==============================] - 1s 2ms/step - loss: 289.7980 - root_mean_squared_error: 16.9148
52/52 [==============================] - 0s 2ms/step - loss: 459.4171 - root_mean_squared_error: 21.3479
52/52 [==============================] - 0s 2ms/step - loss: 604.3700 - root_mean_squared_error: 24.5089

In [16]:
print(X_test_scaled.shape)

(1664, 92)


In [21]:
y_test.shape

(1664, 2)

In [23]:
y_test.head()

,casual,registered
4215,45,186
15865,7,32
3169,18,39
16147,268,383
12565,57,596


In [ ]:
# Get predictions for the testing data
predictions_test = best_model_combined.predict(X_test_scaled)

# Split the predictions into 'casual' and 'registered'
predictions_test_casual = predictions_test[:, 0]
predictions_test_registered = predictions_test[:, 1]

# Get the r^2 for test set
r2_test_casual = r2_score(y_test['casual'], predictions_test_casual)
r2_test_registered = r2_score(y_test['registered'], predictions_test_registered)

# Get predictions for the validation data
predictions_val = best_model_combined.predict(X_val_scaled)

predictions_val_casual = predictions_val[:,0]
predictions_val_registered = predictions_val[:,1]

# Compute the r^2 for validation set
r2_val_casual = r2_score(y_val['casual'], predictions_val_casual)  # Calculate R^2 for 'casual'
r2_val_registered = r2_score(y_val['registered'], predictions_val_registered)  # Calculate R^2 for 'registered'

# Print the R^2 scores
print(f'Test R^2 for casual: {r2_test_casual}')
print(f'Test R^2 for registered: {r2_test_registered}')
print(f'Validation R^2 for casual: {r2_val_casual}')
print(f'Validation R^2 for registered: {r2_val_registered}')

52/52 [==============================] - 0s 2ms/step
[0.92836282 0.96667423]

In [ ]:
import matplotlib.pyplot as plt
# Get the best model
best_model = tuner.get_best_models(num_models=1)[0]

# Retrain the best model and store the history
history = best_model.fit(X_train_scaled, y_train, epochs=30, validation_split=0.2, callbacks=callbacks, batch_size=64)

import matplotlib.pyplot as plt

# Plot the learning curves
plt.figure(figsize=(12, 4))

# Plot the loss
plt.subplot(1, 2, 1)
plt.plot(history.history['loss'], label='Train Loss')
plt.plot(history.history['val_loss'], label='Val Loss')
plt.title('Loss')
plt.legend()

# Plot the root mean squared error
plt.subplot(1, 2, 2)
plt.plot(history.history['root_mean_squared_error'], label='Train RMSE')
plt.plot(history.history['val_root_mean_squared_error'], label='Val RMSE')
plt.title('Root Mean Squared Error')
plt.legend()

plt.show()

# Get Holdout predictions

In [ ]:
# Get the best hyperparameters
best_hp = tuner.get_best_hyperparameters(num_trials=1)[0]

# Print the hyperparameters
print(best_hp.values)

{'num_layers': 2, 'units_0': 160, 'learning_rate': 0.01, 'units_1': 96, 'units_2': 160, 'tuner/epochs': 30, 'tuner/initial_epoch': 0, 'tuner/bracket': 0, 'tuner/round': 0}
best hyperparams 1

{'num_layers': 2, 'units_0': 96, 'learning_rate': 0.01, 'units_1': 160, 'units_2': 160, 'tuner/epochs': 30, 'tuner/initial_epoch': 0, 'tuner/bracket': 0, 'tuner/round': 0}

best hyperparams

In [ ]:
import numpy as np

targets_file = "https://raw.githubusercontent.com/byui-cse/cse450-course/master/data/biking_holdout_test_mini_answers.csv"
targets = pd.read_csv(targets_file)
# Assuming you already have X_mini_scaled
predictions_mini = best_model.predict(X_mini_scaled)
y_mini = targets
# Convert the DataFrame into a numpy array and flatten it
y_mini_array = np.concatenate([y_mini['casual'].values, y_mini['registered'].values])
# Assuming y_mini is the actual target values
df_predictions = pd.DataFrame({'predictions': predictions_mini.flatten()})
df_actual = pd.DataFrame({'actual': y_mini_array})


In [ ]:
from sklearn.metrics import mean_squared_error, r2_score, mean_absolute_error, median_absolute_error
import seaborn as sns
import matplotlib.pyplot as plt
import pandas as pd

student_dict = {}
mse = mean_squared_error(df_actual, df_predictions, squared=False)
student_dict["RMSE"] = mse
student_dict["Mean Abs Error"] = mean_absolute_error(df_actual, df_predictions)
student_dict["Median Abs Error"] = median_absolute_error(df_actual, df_predictions)
student_dict["R2"] = r2_score(df_actual, df_predictions)

combined = pd.concat([df_actual, df_predictions], axis=1)
combined.columns = ["actual", "predictions"]
combined["absdiff"] = (combined["actual"] - combined["predictions"]).abs()
combined["absdiff_pct"] = combined["absdiff"] / combined["actual"]

percents = [5, 10, 20]
for percent in percents:
    student_dict[f"Percent of bikes within {percent} percent"] = len(combined[combined["absdiff_pct"] <= (percent /100)]) / len(combined) * 100

results_df = pd.DataFrame(student_dict, index=[0])
results_df = results_df.round(2)

print(results_df)


In [ ]:
plt.scatter(df_actual['actual'], df_predictions['predictions'])
plt.xlabel('Actual Values')
plt.ylabel('Predicted Values')
plt.title('Actual vs. Predicted Values')
plt.show()

In [ ]:
prediction_error = df_actual['actual'] - df_predictions['predictions']
plt.scatter(df_actual['actual'], prediction_error)
plt.xlabel('Actual Values')
plt.ylabel('Prediction Error')
plt.title('Prediction Error vs. Actual')
plt.axhline(y=0, color='r', linestyle='-')  # Adds a horizontal line at y=0
plt.show()


In [ ]:
import scipy.stats as stats
residuals = df_actual['actual'] - df_predictions['predictions']
stats.probplot(residuals, dist="norm", plot=plt)
plt.title("Normal Q-Q Plot")
plt.show()


In [ ]:
residuals = df_actual['actual'] - df_predictions['predictions']
plt.scatter(df_predictions['predictions'], residuals)
plt.xlabel('Predicted Values')
plt.ylabel('Residuals')
plt.title('Residuals vs. Fitted')
plt.axhline(y=0, color='r', linestyle='-')  # Adds a horizontal line at y=0
plt.show()


In [ ]:
residuals = df_actual['actual'] - df_predictions['predictions']
plt.hist(residuals, bins=30)
plt.xlabel('Residuals')
plt.title('Histogram of Residuals')
plt.show()


In [ ]:
plt.scatter(df_predictions['predictions'], residuals)
plt.xlabel('Predicted Values')
plt.ylabel('Residuals')
plt.title('Residuals vs. Predicted Values')
plt.show()
